Coded using resources from Machine learning mastery with several modifications to suit the problem discussed in the report. (https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/)

The purpose of this code is described in the neural network section of the results in the report. 
It is not entirely finished due to a late discovery of the error in the code we used originally. Hopefully it shows, in combination with the explanation in the report, a more detailed understanding of the process of classifying into multiple classes.

In [35]:
from numpy import array
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from pandas import DataFrame
from matplotlib import pyplot
from keras.losses import CategoricalCrossentropy

import pandas
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

import nltk

from nltk.corpus import wordnet as wn
from nltk.corpus import genesis
genesis_ic = wn.ic(genesis, False, 0.0)

from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.corpus import stopwords
from itertools import repeat

Create a vobaulary list that includes all valid words from the training set.

In [36]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

# turn a doc into clean tokens
def clean_doc(doc):
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', punctuation)
    tokens = [w.translate(table) for w in tokens]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    ps = nltk.wordnet.WordNetLemmatizer()
    tokens = [ps.lemmatize(word) for word in tokens]

    return tokens
    
    
# load all words in a file
def process_vocab(filename, vocab):
    # load doc
    doc = load_doc(filename)
    # clean doc
    tokens = clean_doc(doc)
    # update counts
    vocab.update(tokens)

# load the vocabulary
vocab_filename = 'Project/vocab.txt'
vocab = load_doc(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)
# load all training reviews
caribbean = process_vocab('/Users/akjen/Documents/NTNU/ml/Project/train_lyrics/caribbean.txt', vocab)
christian = process_vocab('/Users/akjen/Documents/NTNU/ml/Project/train_lyrics/christian.txt', vocab)
country = process_vocab('/Users/akjen/Documents/NTNU/ml/Project/train_lyrics/country.txt', vocab)
easylistening = process_vocab('/Users/akjen/Documents/NTNU/ml/Project/train_lyrics/easylistening.txt', vocab)
electronic = process_vocab('/Users/akjen/Documents/NTNU/ml/Project/train_lyrics/electronic.txt', vocab)
folk = process_vocab('/Users/akjen/Documents/NTNU/ml/Project/train_lyrics/folk.txt', vocab)
hiphop = process_vocab('/Users/akjen/Documents/NTNU/ml/Project/train_lyrics/hiphop.txt', vocab)
jazz = process_vocab('/Users/akjen/Documents/NTNU/ml/Project/train_lyrics/jazz.txt', vocab)
latin = process_vocab('/Users/akjen/Documents/NTNU/ml/Project/train_lyrics/latin.txt', vocab)
metal = process_vocab('/Users/akjen/Documents/NTNU/ml/Project/train_lyrics/metal.txt', vocab)
pop = process_vocab('/Users/akjen/Documents/NTNU/ml/Project/train_lyrics/pop.txt', vocab)
punk = process_vocab('/Users/akjen/Documents/NTNU/ml/Project/train_lyrics/punk.txt', vocab)
rnb = process_vocab('/Users/akjen/Documents/NTNU/ml/Project/train_lyrics/rnb.txt', vocab)
rock = process_vocab('/Users/akjen/Documents/NTNU/ml/Project/train_lyrics/rock.txt', vocab)

In [39]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text


# load doc, clean and return line of tokens
def doc_to_line(filename, vocab):
    # load the doc
    doc = load_doc(filename)
    # clean doc
    tokens = clean_doc(doc)
    # filter by vocab 
    tokens = [w for w in tokens if w in vocab]
    return ' '.join(tokens)

# load all docs in a directory
def process_docs(filename, vocab):
    lines = list()
    # walk through all files in the folder
    line = doc_to_line(filename, vocab)
    # add to list
    lines.append(line)
    return lines

# prepare bag of words encoding of docs
def prepare_data(train_docs, mode):
	# create the tokenizer
	tokenizer = Tokenizer()
	# fit the tokenizer on the documents
	tokenizer.fit_on_texts(train_docs)
	# encode training data set
	Xtrain = tokenizer.texts_to_matrix(train_docs, mode=mode)
	return Xtrain

def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(50, input_shape=(n_words,), activation='relu'))
    model.add(Dense(15, input_shape=(n_words,), activation='relu'))
    model.add(Dense(5, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# load all training reviews
caribbean = process_docs('/Users/akjen/Documents/NTNU/ml/Project/train_lyrics/caribbean.txt', vocab)
christian = process_docs('/Users/akjen/Documents/NTNU/ml/Project/train_lyrics/christian.txt', vocab)
country = process_docs('/Users/akjen/Documents/NTNU/ml/Project/train_lyrics/country.txt', vocab)
easylistening = process_docs('/Users/akjen/Documents/NTNU/ml/Project/train_lyrics/easylistening.txt', vocab)
electronic = process_docs('/Users/akjen/Documents/NTNU/ml/Project/train_lyrics/electronic.txt', vocab)
folk = process_docs('/Users/akjen/Documents/NTNU/ml/Project/train_lyrics/folk.txt', vocab)
hiphop = process_docs('/Users/akjen/Documents/NTNU/ml/Project/train_lyrics/hiphop.txt', vocab)
jazz = process_docs('/Users/akjen/Documents/NTNU/ml/Project/train_lyrics/jazz.txt', vocab)
latin = process_docs('/Users/akjen/Documents/NTNU/ml/Project/train_lyrics/latin.txt', vocab)
metal = process_docs('/Users/akjen/Documents/NTNU/ml/Project/train_lyrics/metal.txt', vocab)
pop = process_docs('/Users/akjen/Documents/NTNU/ml/Project/train_lyrics/pop.txt', vocab)
punk = process_docs('/Users/akjen/Documents/NTNU/ml/Project/train_lyrics/punk.txt', vocab)
rnb = process_docs('/Users/akjen/Documents/NTNU/ml/Project/train_lyrics/rnb.txt', vocab)
rock = process_docs('/Users/akjen/Documents/NTNU/ml/Project/train_lyrics/rock.txt', vocab)

train_docs =  country + easylistening + hiphop + latin + pop
# prepare data for mode
Xtrain = prepare_data(train_docs, 'tfidf')

ytrain = []
# prepare labels
ytrain = array(['country' for _ in range(len(country))] + ['easylistening' for _ in range(len(easylistening))] + ['hiphop' for _ in range(len(hiphop))] + ['latin' for _ in range(len(latin))] + ['pop' for _ in range(len(pop))])
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(ytrain)
encoded_Y = encoder.transform(ytrain)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, Xtrain, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))



ValueError: Cannot have number of splits n_splits=10 greater than the number of samples: n_samples=5.

In [ ]:

# define baseline model
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(8, input_dim=4, activation='relu'))
	model.add(Dense(3, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

## All genres except caribbean
christian + country + easylistening + electronic + folk + hiphop + jazz + latin + metal + pop + punk + rnb + rock


## Removed the worst performing categories in kNN
country + easylistening + hiphop + latin + pop + punk + rnb + rock

## Including only best performing categories for kNN
country + easylistening + hiphop + latin + pop

In [ ]:
ytrain = array([0 for _ in range(len(christian))] + [1 for _ in range(len(country))] + [2 for _ in range(len(easylistening))] + [3 for _ in range(len(electronic))] + [4 for _ in range(len(folk))] +
               [5 for _ in range(len(hiphop))] + [6 for _ in range(len(jazz))] + [7 for _ in range(len(latin))] + [8 for _ in range(len(metal))] + [9 for _ in range(len(pop))] + [10 for _ in range(len(punk))] + 
               [11 for _ in range(len(rnb))] + [12 for _ in range(len(rock))])
ytest = array([0 for _ in range(len(tchristian))] + [1 for _ in range(len(tcountry))] + [2 for _ in range(len(teasylistening))] + [3 for _ in range(len(telectronic))] + [4 for _ in range(len(tfolk))] +
               [5 for _ in range(len(thiphop))] + [6 for _ in range(len(tjazz))] + [7 for _ in range(len(tlatin))] + [8 for _ in range(len(tmetal))] + [9 for _ in range(len(tpop))] + [10 for _ in range(len(tpunk))] + 
               [11 for _ in range(len(trnb))] + [12 for _ in range(len(trock))])